In [ ]:
import cv2
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
def load_labels(data_dir):
    """
    Dynamically load labels based on folder structure.
    :param data_dir: Path to the training images directory.
    :return: A dictionary mapping IDs to labels.
    
    """
    labels = {}
    for idx, folder_name in enumerate(sorted(os.listdir(data_dir)), start=1):
        if os.path.isdir(os.path.join(data_dir, folder_name)):
            labels[idx] = folder_name
    return labels

In [ ]:
def prepare_data(data_dir):
    """
    Prepare training data from a directory with subfolders for each person.
    
    :param data_dir: Path to the parent directory containing subfolders for each person.
    :return: Tuple of faces and IDs.
    """
    faces = []
    ids = []
    
    # Iterate through each subfolder (person's name)
    for person_name in os.listdir(data_dir):
        person_folder = os.path.join(data_dir, person_name)
        
        if os.path.isdir(person_folder):  # Ensure it's a directory
            # Iterate through each image in the person's folder
            for image_name in os.listdir(person_folder):
                image_path = os.path.join(person_folder, image_name)
                
                try:
                    # Read the image, convert to grayscale, and prepare it for training
                    img = Image.open(image_path).convert('L')  # Convert to grayscale
                    image_np = np.array(img, 'uint8')  # Convert to NumPy array
                    
                    # Use the folder index as the ID (or customize if needed)
                    id = list(os.listdir(data_dir)).index(person_name) + 1
                    
                    faces.append(image_np)
                    ids.append(id)
                except Exception as e:
                    print(f"Error processing {image_path}: {e}")
                    
    return faces, np.array(ids)

In [ ]:
def augment_data(faces, ids):
    """
    Perform data augmentation on the training dataset.
    
    :param faces: List of face images (NumPy arrays).
    :param ids: Array of corresponding IDs for the face images.
    :return: Augmented faces and IDs.
    """
    augmented_faces = []
    augmented_ids = []

    for face, id in zip(faces, ids):
        # Original image
        augmented_faces.append(face)
        augmented_ids.append(id)

        # Flip horizontally
        flipped_face = cv2.flip(face, 1)
        augmented_faces.append(flipped_face)
        augmented_ids.append(id)

        # Add Gaussian blur
        blurred_face = cv2.GaussianBlur(face, (5, 5), 0)
        augmented_faces.append(blurred_face)
        augmented_ids.append(id)

        # Add brightness and contrast adjustment
        adjusted_face = cv2.convertScaleAbs(face, alpha=1.2, beta=30)
        augmented_faces.append(adjusted_face)
        augmented_ids.append(id)

    return augmented_faces, np.array(augmented_ids)

In [ ]:
def train_test_split_data(faces, ids, test_size=0.2):
    """
    Split the data into training and testing sets.
    :param faces: List of face images (NumPy arrays).
    :param ids: Array of corresponding IDs for the face images.
    :param test_size: Proportion of the dataset to include in the test split.
    :return: Training and testing sets (faces and ids).
    """
    faces_train, faces_test, ids_train, ids_test = train_test_split(faces, ids, test_size=test_size, random_state=42)
    return faces_train, faces_test, ids_train, ids_test

In [ ]:
def train_classifier(faces, ids, output_model="Yunetclassifier.xml"):
    print("Training classifier...")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write(output_model)
    print(f"Training complete. Model saved as {output_model}.")

In [ ]:
def calculate_accuracy(faces_test, ids_test):
    """
    Calculate the accuracy of the classifier on the test set.
    :param clf: Trained classifier.
    :param faces_test: Test set faces.
    :param ids_test: True labels for the test set.
    :return: Accuracy score.
    """
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("Yunetclassifier.xml")
    predictions = []
    for face in faces_test:
        label, _ = clf.predict(face)
        predictions.append(label)
    accuracy = accuracy_score(ids_test, predictions)
    return accuracy

In [ ]:
def predict_webcam(classifier, face_detector, labels):
    """
    Capture video from webcam and predict faces in real-time using the trained classifier and ONNX-based face detector.

    :param classifier: Trained LBPH face recognizer model.
    :param face_detector: ONNX-based face detector.
    :param labels: Dictionary mapping IDs to labels.
    """
    print("Starting webcam...")

    video_capture = cv2.VideoCapture(0)

    while True:
        ret, img = video_capture.read()
        if not ret:
            print("Failed to capture image.")
            break

        # Prepare image for the face detector
        height, width, _ = img.shape
        face_detector.setInputSize((width, height))

        # Detect faces
        _, faces = face_detector.detect(img)

        if faces is not None:
            for face in faces:
                x, y, w, h = map(int, face[:4])  # Bounding box coordinates
                face_roi = img[y:y+h, x:x+w]
                gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)

                # Predict label
                id, confidence = classifier.predict(gray_face)
                confidence = int(100 * (1 - confidence / 300))

                if confidence > 70 and id in labels:
                    label = labels.get(id, "UNKNOWN")
                    color = (0, 255, 0)  # Green for recognized faces
                    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.putText(img, f"{label}", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                else:
                    label = "UNKNOWN"
                    color = (0, 0, 255)  # Red for unknown faces
                    cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
                    cv2.putText(img, label, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)

            # Draw the bounding box around the face
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)

                

        # Show the webcam feed with predictions
        cv2.imshow("Webcam - Face Detection", img)

        # Exit on pressing 'Esc' key
        if cv2.waitKey(1) == 27:  # 'Esc' key
            break

    video_capture.release()
    cv2.destroyAllWindows()

In [ ]:
def predict_single_image(classifier, face_detector, labels, image_path):
    """
    Predict the face in a single image using the trained classifier and FaceDetectorYN.

    :param classifier: Trained LBPH face recognizer model.
    :param face_detector: OpenCV FaceDetectorYN object.
    :param labels: Dictionary mapping IDs to labels.
    :param image_path: Path to the input image.
    """
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load image: {image_path}")
        return

    # Convert to grayscale (LBPH requires grayscale input)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces using FaceDetectorYN
    height, width = img.shape[:2]
    face_detector.setInputSize((width, height))  # Update the input size for detection
    _, faces = face_detector.detect(img)

    if faces is not None and len(faces) > 0:
        for face in faces:
            x, y, w, h = map(int, face[:4])  # Extract bounding box coordinates
            face_roi = gray_img[y:y+h, x:x+w]

            # Predict label
            id, confidence = classifier.predict(face_roi)

            # Determine label based on confidence
            if confidence > 50 and id in labels:
                label = labels.get(id, "UNKNOWN")
                color = (0, 255, 0)  # Green for recognized faces
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
                cv2.putText(img, f"{label}", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
            else:
                label = "UNKNOWN"
                color = (0, 0, 255)  # Red for unknown faces
                cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
                cv2.putText(img, label, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)

            # Draw the bounding box around the face
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
    else:
        print("No faces detected.")

    # Display the result (optional)
    cv2.imshow("Prediction", img)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [ ]:
# Paths for training data
training_data_dir = input("Enter the Folder Name")  # Folder containing training images
data_dir = training_data_dir

labels = load_labels(data_dir)
print("Labels loaded:", labels)

# Step 1: Load Training Data
faces, ids = prepare_data(training_data_dir)

# Step 2: Augment Data
faces, ids = augment_data(faces, ids)
print(f"Data augmented. Total samples: {len(faces)}")

# Step 3: Train-Test Split
faces_train, faces_test, ids_train, ids_test = train_test_split_data(faces, ids)
print(f"Training set size: {len(faces_train)}, Testing set size: {len(faces_test)}")

onnx_model_path = "face_detection_yunet_2023mar.onnx"
# Step 1: Load Training Data
faces, ids = prepare_data(training_data_dir)

# Step 3: Train Classifier
train_classifier(faces, ids, "Yunetclassifier.xml")

# Step 4: Load ONNX Face Detector
face_detector = cv2.FaceDetectorYN.create(onnx_model_path, "", (320, 320), 0.9, 0.3, 5000)
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("Yunetclassifier.xml")

# Step 5: Calculate Accuracy
accuracy = calculate_accuracy(faces_test, ids_test)
print(f"Model accuracy on test data: {accuracy * 100:.2f}%")

In [ ]:
predict_webcam(clf, face_detector, labels)